
# Notebook A — MNIST Stacked Ensemble (VGG16 · VGG19 · ResNet50 + Meta LR)

**Paper Companion:** *Optimizing Handwritten Digit Recognition: A Novel Stacked CNN Ensemble for Enhanced Feature Extraction*  

**What this notebook does**  
- Small, **balanced subset** for speed (adjustable)  
- **Transfer learning** with explicit preprocess + **penultimate stacking**  
- **50 epochs** per base model (as requested, for consistent learning curves)  
- **k-fold CV** with multiple seeds + **95% CI**  
- **Significance tests** (Wilcoxon, McNemar)  
- **Figures** exported for the paper (training curves, feature maps, confusion matrix, CV plots)  
- **Colab-ready** (includes a quick `%pip install` cell)


In [ ]:

# ===========================
# 0) Setup (Colab / first run)
# ===========================
# If running on Colab (or first time locally), uncomment this cell:
# %pip install -q tensorflow tensorflow-datasets scikit-learn matplotlib pandas numpy scipy


In [ ]:

# ================================================================
# Notebook A: MNIST Stacked Ensemble (VGG16/VGG19/ResNet50 + Meta LR)
# - Small balanced subset for speed (adjustable)
# - Transfer learning + penultimate stacking
# - k-fold CV with multiple seeds + 95% CI
# - Significance tests (Wilcoxon, McNemar)
# - Plots & exports for paper
# Colab-ready, TF 2.x
# ================================================================

# ---------------------------
# 1) Imports & global config
# ---------------------------
import os, time, math, json, numpy as np, tensorflow as tf, matplotlib.pyplot as plt
import pandas as pd
from pathlib import Path

from tensorflow.keras import layers, Model
from tensorflow.keras.applications import VGG16, VGG19, ResNet50
from tensorflow.keras.optimizers import Adam

from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix, ConfusionMatrixDisplay

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.keras.utils.set_random_seed(SEED)

# Output folders
OUT = Path("paper_outputs_mnist")
OUT.mkdir(exist_ok=True, parents=True)

# ---------------------------
# 2) Hyperparameters & logging
# ---------------------------
HP = {
    "dataset": "MNIST",
    "num_classes": 10,
    "img_size": 224,
    "batch_size": 8,
    "train_subset_n": 500,    # balanced total; increase for final runs
    "test_subset_n": 100,     # balanced total
    "epochs_per_model": 50,   # <-- set to 50 to reproduce long learning curves
    "fine_tune_last_layers": {"vgg16": 4, "vgg19": 4, "resnet50": 10},
    "optimizer": "Adam",
    "learning_rate": 1e-4,
    "augment": {
        "random_flip": "HORIZONTAL",   # none / horizontal / vertical
        "random_rotation": 0.08,        # ~4.5 degrees
        "random_zoom": 0.08,
    },
    "cv_kfolds": 3,            # use 5 for final paper
    "cv_seeds": [42, 43],      # use 5 seeds for final paper: [42,43,44,45,46]
}
with open(OUT / "hyperparams.json", "w") as f:
    json.dump(HP, f, indent=2)

# Device info
dev_info = {
    "tf_version": tf.__version__,
    "gpus": [x.name for x in tf.config.list_physical_devices('GPU')],
    "cpus": [x.name for x in tf.config.list_physical_devices('CPU')],
}
with open(OUT / "device_info.json", "w") as f:
    json.dump(dev_info, f, indent=2)
print("Device info:", dev_info)


In [ ]:

# ---------------------------------------------
# 3) Load MNIST & build small balanced subsets
# ---------------------------------------------
(x_train_full, y_train_full), (x_test_full, y_test_full) = tf.keras.datasets.mnist.load_data()

def make_balanced_subset(x, y, total_count, num_classes=10, seed=SEED):
    rng = np.random.default_rng(seed)
    per_class = max(1, total_count // num_classes)
    xs, ys = [], []
    for c in range(num_classes):
        idx = np.where(y == c)[0]
        pick = rng.choice(idx, size=per_class, replace=False)
        xs.append(x[pick]); ys.append(y[pick])
    return np.concatenate(xs), np.concatenate(ys)

x_train, y_train = make_balanced_subset(x_train_full, y_train_full, HP["train_subset_n"])
x_test,  y_test  = make_balanced_subset(x_test_full,  y_test_full,  HP["test_subset_n"])


In [ ]:

# ---------------------------------------------
# 4) Preprocess & augmentation (explicit params)
# ---------------------------------------------
IMG_SIZE = HP["img_size"]
NUM_CLASSES = HP["num_classes"]
BATCH_SIZE = HP["batch_size"]

# augmentation layer (for training only)
data_augment = tf.keras.Sequential(name="augment")
if HP["augment"]["random_flip"].upper() == "HORIZONTAL":
    data_augment.add(layers.RandomFlip("horizontal"))
elif HP["augment"]["random_flip"].upper() == "VERTICAL":
    data_augment.add(layers.RandomFlip("vertical"))
# rotations & zooms:
data_augment.add(layers.RandomRotation(HP["augment"]["random_rotation"]))
data_augment.add(layers.RandomZoom(HP["augment"]["random_zoom"]))

def preprocess(image, label, training=False):
    image = tf.expand_dims(image, -1)
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    image = tf.image.grayscale_to_rgb(image)
    image = tf.cast(image, tf.float32) / 255.0
    if training:
        image = data_augment(image)
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label

SHUF = min(len(x_train), 1000)
train_ds = (tf.data.Dataset.from_tensor_slices((x_train, y_train))
            .shuffle(SHUF, seed=SEED)
            .map(lambda i,l: preprocess(i,l, True), num_parallel_calls=tf.data.AUTOTUNE)
            .batch(BATCH_SIZE)
            .prefetch(tf.data.AUTOTUNE))

val_ds = (tf.data.Dataset.from_tensor_slices((x_test, y_test))
          .map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
          .batch(BATCH_SIZE)
          .prefetch(tf.data.AUTOTUNE))

for imgs, labs in train_ds.take(1):
    print("Train batch:", imgs.shape, labs.shape)


In [ ]:

# Class distribution plots (for paper)
def plot_class_distribution(labels, title, save_path):
    counts = np.bincount(labels, minlength=NUM_CLASSES)
    plt.figure(figsize=(6,4))
    plt.bar(np.arange(NUM_CLASSES), counts)
    plt.title(title); plt.xlabel("Digit"); plt.ylabel("Count")
    plt.xticks(np.arange(NUM_CLASSES)); plt.grid(axis='y', alpha=0.3)
    plt.tight_layout(); plt.savefig(save_path, dpi=300); plt.show()

plot_class_distribution(y_train, "Training Set Class Distribution (subset)", OUT/"class_dist_train.png")
plot_class_distribution(y_test,  "Testing Set Class Distribution (subset)",  OUT/"class_dist_test.png")


In [ ]:

# -------------------------------------------------------
# 5) Build base models (VGG16, VGG19, ResNet50)
#    - include explicit preprocess_input
#    - unfreeze top-k layers for light fine-tuning
# -------------------------------------------------------
def build_base_model(base_cls, fine_tune_at, lr=HP["learning_rate"], name_tag=""):
    # choose preprocess according to backbone
    if base_cls in [VGG16, VGG19]:
        preprocess_fn = tf.keras.applications.vgg16.preprocess_input
    elif base_cls is ResNet50:
        preprocess_fn = tf.keras.applications.resnet50.preprocess_input
    else:
        raise ValueError("Unsupported backbone")

    inp = layers.Input(shape=(IMG_SIZE, IMG_SIZE, 3), name='input_rgb_0to1')
    x   = layers.Lambda(preprocess_fn, name='imagenet_preprocess')(inp)
    base = base_cls(weights='imagenet', include_top=False, input_tensor=x)

    for lay in base.layers: lay.trainable = False
    for lay in base.layers[-fine_tune_at:]: lay.trainable = True

    # head
    y = layers.GlobalAveragePooling2D(name='gap')(base.output)
    y = layers.Dropout(0.5, name='drop1')(y)
    y = layers.Dense(256, activation='relu', name='fc1')(y)
    y = layers.Dropout(0.5, name='drop2')(y)
    out= layers.Dense(NUM_CLASSES, activation='softmax', name='predictions')(y)

    model = Model(inputs=inp, outputs=out, name=f"{base_cls.__name__}_HDR{name_tag}")
    model.compile(optimizer=Adam(learning_rate=lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

vgg16_model    = build_base_model(VGG16,  HP["fine_tune_last_layers"]["vgg16"])
vgg19_model    = build_base_model(VGG19,  HP["fine_tune_last_layers"]["vgg19"])
resnet50_model = build_base_model(ResNet50,HP["fine_tune_last_layers"]["resnet50"])

# Parameter counts (for computational reporting)
comp_report = {}
for name, m in [("vgg16", vgg16_model), ("vgg19", vgg19_model), ("resnet50", resnet50_model)]:
    comp_report[name] = {"trainable_params": int(np.sum([np.prod(v.shape) for v in m.trainable_weights])),
                         "non_trainable_params": int(np.sum([np.prod(v.shape) for v in m.non_trainable_weights]))}
with open(OUT/"param_counts.json", "w") as f: json.dump(comp_report, f, indent=2)
print("Param counts:", comp_report)

# Optional FLOPs estimation (may fail depending on TF build; ignored if it does)
def try_estimate_flops(model):
    try:
        concrete = tf.function(model).get_concrete_function(tf.TensorSpec([1, IMG_SIZE, IMG_SIZE, 3], tf.float32))
        from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2_as_graph
        frozen_func, graph_def = convert_variables_to_constants_v2_as_graph(concrete)
        run_meta = tf.compat.v1.RunMetadata()
        opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()
        flops = tf.compat.v1.profiler.profile(graph=frozen_func.graph, run_meta=run_meta, cmd='op', options=opts)
        return int(flops.total_float_ops)
    except Exception:
        return None

for name, m in [("vgg16", vgg16_model), ("vgg19", vgg19_model), ("resnet50", resnet50_model)]:
    fl = try_estimate_flops(m)
    comp_report[name]["approx_FLOPs"] = fl
with open(OUT/"param_counts.json", "w") as f: json.dump(comp_report, f, indent=2)


In [ ]:

# -------------------------------------------------------
# 6) Train base models (50 epochs) & save learning curves
# -------------------------------------------------------
def train_and_time(model, train_ds, val_ds, epochs, tag):
    t0 = time.time()
    hist = model.fit(train_ds, epochs=epochs, validation_data=val_ds, verbose=2)
    dt = time.time() - t0
    # save history CSV
    pd.DataFrame(hist.history).to_csv(OUT/f"history_{tag}.csv", index=False)
    return hist, dt

histories = {}
train_times = {}
for name, model in [("vgg16", vgg16_model), ("vgg19", vgg19_model), ("resnet50", resnet50_model)]:
    print(f"\nTraining {name} ...")
    h, dt = train_and_time(model, train_ds, val_ds, HP["epochs_per_model"], name)
    histories[name] = h
    train_times[name] = dt
with open(OUT/"train_times_seconds.json","w") as f: json.dump(train_times, f, indent=2)
print("Train times (sec):", train_times)

# 3×2 training curves figure
def _plot_pair(ax_acc, ax_loss, history, title_prefix):
    ep = range(1, len(history.history['accuracy'])+1)
    ax_acc.plot(ep, history.history['accuracy'], marker='o', ms=3, lw=1.5, label='Training Acc')
    ax_acc.plot(ep, history.history['val_accuracy'], marker='*', ms=3, lw=1.5, ls='--', label='Validation Acc')
    ax_acc.set_title(f'Training and Validation Accuracy for {title_prefix} Model', fontsize=9)
    ax_acc.set_xlabel('Epochs', fontsize=8); ax_acc.set_ylabel('Accuracy', fontsize=8); ax_acc.grid(True, alpha=0.25); ax_acc.legend(loc='lower right', fontsize=7)

    ax_loss.plot(ep, history.history['loss'], marker='o', ms=3, lw=1.5, label='Training Loss')
    ax_loss.plot(ep, history.history['val_loss'], marker='*', ms=3, lw=1.5, ls='--', label='Validation Loss')
    ax_loss.set_title('Training and Validation Loss', fontsize=9)
    ax_loss.set_xlabel('Epochs', fontsize=8); ax_loss.set_ylabel('Loss', fontsize=8); ax_loss.grid(True, alpha=0.25); ax_loss.legend(loc='upper right', fontsize=7)

def plot_3x2(histories, save_path):
    fig, axes = plt.subplots(3,2, figsize=(8.5,11))
    _plot_pair(axes[0,0], axes[0,1], histories["vgg16"], "VGG16")
    _plot_pair(axes[1,0], axes[1,1], histories["vgg19"], "VGG19")
    _plot_pair(axes[2,0], axes[2,1], histories["resnet50"], "ResNet50")
    for r in axes:
        for a in r: a.tick_params(labelsize=8)
    plt.tight_layout(pad=1.2); plt.savefig(save_path, dpi=300, bbox_inches='tight'); plt.show()

plot_3x2(histories, OUT/"training_curves_3x2.png")


In [ ]:

# -------------------------------------------------------
# 7) Feature maps: first 8 maps from VGG16 block1_conv1
# -------------------------------------------------------
from tensorflow.keras.applications.vgg16 import preprocess_input
sample = x_train[0]
img = tf.image.resize(tf.expand_dims(sample, -1), [IMG_SIZE, IMG_SIZE])
img = tf.image.grayscale_to_rgb(img)
img_b = tf.expand_dims(tf.cast(img, tf.float32), 0) / 255.0
img_b_pp = preprocess_input(img_b)  # VGG16 preprocess

layer_model = tf.keras.Model(inputs=vgg16_model.input,
                             outputs=vgg16_model.get_layer('block1_conv1').output)
fmaps = layer_model.predict(img_b_pp, verbose=0)

fig, axes = plt.subplots(1,8, figsize=(20,5))
for i in range(8):
    axes[i].imshow(fmaps[0,:,:,i], cmap='viridis'); axes[i].axis('off')
plt.suptitle("First 8 Feature Maps from block1_conv1 (VGG16)", fontsize=14)
plt.tight_layout(); plt.savefig(OUT/"feature_maps_block1conv1_vgg16.png", dpi=300); plt.show()


In [ ]:

# -------------------------------------------------------
# 8) Evaluate base models; build penultimate features + Meta-learner
# -------------------------------------------------------
def to_rgb_array(x_uint8):
    x = tf.convert_to_tensor(x_uint8)
    x = tf.expand_dims(x, -1)
    x = tf.image.resize(x, [IMG_SIZE, IMG_SIZE])
    x = tf.image.grayscale_to_rgb(x)
    x = tf.cast(x, tf.float32) / 255.0
    return x.numpy()

x_test_rgb = to_rgb_array(x_test)
y_test_1h = tf.one_hot(y_test, NUM_CLASSES)

acc_vgg16   = vgg16_model.evaluate(x_test_rgb, y_test_1h, verbose=0)[1]
acc_vgg19   = vgg19_model.evaluate(x_test_rgb, y_test_1h, verbose=0)[1]
acc_resnet  = resnet50_model.evaluate(x_test_rgb, y_test_1h, verbose=0)[1]
print(f"Test Acc — VGG16: {acc_vgg16:.4f}  VGG19: {acc_vgg19:.4f}  ResNet50: {acc_resnet:.4f}")

def build_penultimate(model, layer_name='gap'):
    return tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

feat_vgg16   = build_penultimate(vgg16_model,  'gap')
feat_vgg19   = build_penultimate(vgg19_model,  'gap')
feat_resnet  = build_penultimate(resnet50_model,'gap')

def predict_feats(fm, x, bs=64):
    arr = []
    for i in range(0, len(x), bs):
        arr.append(fm.predict(x[i:i+bs], verbose=0))
    return np.vstack(arr)

x_train_rgb = to_rgb_array(x_train)
F_tr = [predict_feats(fm, x_train_rgb) for fm in [feat_vgg16, feat_vgg19, feat_resnet]]
F_te = [predict_feats(fm, x_test_rgb)  for fm in [feat_vgg16, feat_vgg19, feat_resnet]]
X_tr = np.concatenate(F_tr, axis=1)
X_te = np.concatenate(F_te, axis=1)

scaler = StandardScaler()
X_tr_s = scaler.fit_transform(X_tr)
X_te_s = scaler.transform(X_te)

meta = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
meta.fit(X_tr_s, y_train)
y_pred_meta = meta.predict(X_te_s)
meta_acc = accuracy_score(y_test, y_pred_meta)
print(f"Meta-learner (hold-out test) accuracy: {meta_acc:.4f}")

# Meta-learner accuracy over iterations (SGD)
classes = np.unique(y_train)
sgd = SGDClassifier(loss="log_loss", max_iter=1, learning_rate="optimal", tol=None, random_state=SEED, warm_start=True)
acc_iters = []
EPOCHS_SGD = 30
sgd.partial_fit(X_tr_s, y_train, classes=classes)
acc_iters.append(accuracy_score(y_test, sgd.predict(X_te_s)))
for _ in range(EPOCHS_SGD-1):
    sgd.partial_fit(X_tr_s, y_train)
    acc_iters.append(accuracy_score(y_test, sgd.predict(X_te_s)))

plt.figure(figsize=(7,4))
plt.plot(range(1, EPOCHS_SGD+1), acc_iters, marker='o', lw=2)
plt.title("Meta-Learner Accuracy Over Iterations"); plt.xlabel("Iteration"); plt.ylabel("Accuracy")
plt.grid(True, alpha=0.3); plt.tight_layout(); plt.savefig(OUT/"meta_accuracy_over_iterations.png", dpi=300); plt.show()

# Comparison bar chart
labels = ["VGG16","VGG19","ResNet50","Meta-LR"]
scores = [acc_vgg16, acc_vgg19, acc_resnet, meta_acc]
plt.figure(figsize=(6.5,4))
bars = plt.bar(labels, scores)
for b, s in zip(bars, scores):
    plt.text(b.get_x()+b.get_width()/2, b.get_height()+0.005, f"{s:.3f}", ha='center', va='bottom', fontsize=10)
plt.ylim(0,1.0); plt.ylabel("Accuracy"); plt.title("Model Accuracy Comparison (hold-out test)")
plt.grid(axis='y', alpha=0.2); plt.tight_layout(); plt.savefig(OUT/"model_accuracy_comparison_holdout.png", dpi=300); plt.show()

# Confusion matrix
cm = confusion_matrix(y_test, y_pred_meta, labels=np.arange(NUM_CLASSES))
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=np.arange(NUM_CLASSES))
fig, ax = plt.subplots(figsize=(6,5))
disp.plot(ax=ax, cmap='Blues', colorbar=True, values_format='d')
plt.title("Confusion Matrix for the Meta-Learner")
plt.tight_layout(); plt.savefig(OUT/"meta_confusion_matrix.png", dpi=300); plt.show()


In [ ]:

# -------------------------------------------------------
# 9) Cross-validation + multi-seed + CI + significance tests
# -------------------------------------------------------
def compute_ci(accs, alpha=0.05):
    m  = float(np.mean(accs))
    sd = float(np.std(accs, ddof=1)) if len(accs)>1 else 0.0
    z  = 1.96
    half = z*sd/math.sqrt(len(accs)) if len(accs)>1 else 0.0
    return m, sd, (m-half, m+half)

def to_rgb_array(x_uint8):
    x = tf.convert_to_tensor(x_uint8)
    x = tf.expand_dims(x, -1)
    x = tf.image.resize(x, [IMG_SIZE, IMG_SIZE])
    x = tf.image.grayscale_to_rgb(x)
    x = tf.cast(x, tf.float32) / 255.0
    return x.numpy()

def build_penultimate(model, layer_name='gap'):
    return tf.keras.Model(inputs=model.input, outputs=model.get_layer(layer_name).output)

def predict_feats(fm, x, bs=64):
    arr = []
    for i in range(0, len(x), bs):
        arr.append(fm.predict(x[i:i+bs], verbose=0))
    return np.vstack(arr)

def run_cv_multiseed(models_dict, x_uint8, y, seeds, kfolds):
    results = {k: {'accs': [], 'preds': [], 'truths': []} for k in list(models_dict.keys()) + ['meta']}
    for seed in seeds:
        skf = StratifiedKFold(n_splits=kfolds, shuffle=True, random_state=seed)
        x_rgb = to_rgb_array(x_uint8)
        feat_models = {n: build_penultimate(m,'gap') for n,m in models_dict.items()}
        for (i_tr, i_va) in skf.split(x_uint8, y):
            x_tr, y_tr = x_rgb[i_tr], y[i_tr]
            x_va, y_va = x_rgb[i_va], y[i_va]

            # base
            for name, model in models_dict.items():
                yhat = np.argmax(model.predict(x_va, verbose=0), axis=1)
                acc  = accuracy_score(y_va, yhat)
                results[name]['accs'].append(acc)
                results[name]['preds'].append(yhat)
                results[name]['truths'].append(y_va)

            # meta
            F_tr = [predict_feats(feat_models[n], x_tr) for n in models_dict.keys()]
            F_va = [predict_feats(feat_models[n], x_va) for n in models_dict.keys()]
            X_tr = np.concatenate(F_tr, axis=1); X_va = np.concatenate(F_va, axis=1)
            sc   = StandardScaler(); X_tr_s = sc.fit_transform(X_tr); X_va_s = sc.transform(X_va)
            clf  = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=1000)
            clf.fit(X_tr_s, y_tr); yhat_m = clf.predict(X_va_s); acc_m = accuracy_score(y_va, yhat_m)
            results['meta']['accs'].append(acc_m); results['meta']['preds'].append(yhat_m); results['meta']['truths'].append(y_va)

    for name in results:
        results[name]['preds']  = np.concatenate(results[name]['preds'])
        results[name]['truths'] = np.concatenate(results[name]['truths'])
    return results

models_dict = {"vgg16": vgg16_model, "vgg19": vgg19_model, "resnet50": resnet50_model}
cv_results  = run_cv_multiseed(models_dict, x_train, y_train, HP["cv_seeds"], HP["cv_kfolds"])

# summary table
rows = []
for name in ['vgg16','vgg19','resnet50','meta']:
    m, sd, (lo, hi) = compute_ci(cv_results[name]['accs'])
    rows.append({"Model": name, "MeanAcc": m, "Std": sd, "CI95_Low": lo, "CI95_High": hi, "N": len(cv_results[name]['accs'])})
df_cv = pd.DataFrame(rows)
df_cv.to_csv(OUT/"cv_summary.csv", index=False)
print(df_cv)

# Significance tests
from scipy.stats import wilcoxon
def wilcoxon_vs_meta(cv_res, base_name):
    a = np.array(cv_res['meta']['accs']); b = np.array(cv_res[base_name]['accs'])
    stat, p = wilcoxon(a, b, zero_method='wilcox', correction=True, alternative='greater')
    return float(stat), float(p)

def mcnemar_test(truth, pred_a, pred_b):
    n01 = np.sum((pred_a == truth) & (pred_b != truth))
    n10 = np.sum((pred_a != truth) & (pred_b == truth))
    n = n01 + n10
    if n == 0: return 0.0, 1.0
    chi2 = (abs(n01 - n10) - 1)**2 / n
    try:
        from scipy.stats import chi2 as chi2dist
        p = float(chi2dist.sf(chi2, df=1))
    except Exception:
        p = np.exp(-0.5*chi2)
    return float(chi2), float(p)

sig_rows = []
for base in ['vgg16','vgg19','resnet50']:
    W, p = wilcoxon_vs_meta(cv_results, base)
    sig_rows.append({"Test":"Wilcoxon", "Comparison": f"meta > {base}", "W": W, "p": p})

best_base = max(['vgg16','vgg19','resnet50'], key=lambda k: np.mean(cv_results[k]['accs']))
chi2, p = mcnemar_test(cv_results['meta']['truths'], cv_results['meta']['preds'], cv_results[best_base]['preds'])
sig_rows.append({"Test":"McNemar", "Comparison": f"meta vs {best_base}", "chi2": chi2, "p": p})
pd.DataFrame(sig_rows).to_csv(OUT/"significance_tests.csv", index=False)
print(pd.DataFrame(sig_rows))

# Boxplot per fold×seed
plt.figure(figsize=(6.5,4))
plt.boxplot([cv_results['vgg16']['accs'], cv_results['vgg19']['accs'], cv_results['resnet50']['accs'], cv_results['meta']['accs']],
            labels=['VGG16','VGG19','ResNet50','Meta-LR'], showmeans=True)
plt.ylabel("Accuracy"); plt.title("Cross-validated Accuracies (per fold × seed)"); plt.grid(axis='y', alpha=0.3)
plt.tight_layout(); plt.savefig(OUT/"cv_boxplot.png", dpi=300); plt.show()

# Bar chart of CV mean±std
means = df_cv["MeanAcc"].values; stds = df_cv["Std"].values; labels = df_cv["Model"].values
plt.figure(figsize=(6.5,4))
bars = plt.bar(labels, means, yerr=stds, capsize=4)
for b, s in zip(bars, means):
    plt.text(b.get_x()+b.get_width()/2, b.get_height()+0.005, f"{s:.3f}", ha='center', va='bottom', fontsize=10)
plt.ylim(0,1.0); plt.ylabel("Accuracy"); plt.title("Model Accuracy Comparison (CV mean ± std)")
plt.grid(axis='y', alpha=0.2); plt.tight_layout(); plt.savefig(OUT/"model_accuracy_comparison_cv.png", dpi=300); plt.show()

print("\\nAll MNIST outputs saved under:", OUT.resolve())
